# 2. Search Engine

### Import libraries

In [87]:
import json
import numpy as np
import pandas as pd
import os
import csv
import itertools
import re
import csv
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from itertools import chain
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import cosine_similarity    
import operator

In [88]:
Dict = {}
ps = PorterStemmer()
attr = ['Intro', 'Plot']
stop = set(stopwords.words('english'))

In [89]:
path = '/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/tsv_files/'

## 2.1.1) Create your index!

This part has been done in the previous part, directly while creating the tsv files.

### Open and remove duplicates from the Initial dictionary - Save

In [4]:
f =  open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/Good.json', 'r')
Dict = json.load(f)

In [5]:
good_dict = {}
for i in list([*Dict]):
    good_dict[i] = list(set(Dict[i]))

In [6]:
with open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/Good_Good.json', 'w') as outfile:
    json.dump(good_dict, outfile)

### Load the Initial dictionary

In [8]:
f =  open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/Good_Good.json', 'r')
good_dict = json.load(f)

## 2.1.2) Execute the query

### Selection of the documents containg all the words in the query

In [94]:
lst = []
for word in query:
    s = []
    docs = New_Dict.get(word)
    if docs is None or len(docs) == 0:
        continue
    else:
        for i in docs:
            s.append(i[0])
    lst.append(s)
result = list(set(lst[0]).intersection(*lst))


['movie8293.tsv',
 'movie3907.tsv',
 'movie7159.tsv',
 'movie6545.tsv',
 'movie7745.tsv',
 'movie3235.tsv',
 'movie9006.tsv',
 'movie9326.tsv']

## 2.2) Conjunctive query & Ranking score

### 2.2.1) Inverted index

In [13]:
path = '/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/tsv_files/'
attr = ['Intro', 'Plot']  
ps = PorterStemmer()
stop = set(stopwords.words('english'))
lst = list(Dict.keys())
len_dict = len(os.listdir(path))
New_Dict = {}

In [14]:
i = []
for word in good_dict:
    val_word = list(good_dict[word])
    for doc in good_dict[word]:
        file = []
        s=[]
        with open(path+doc) as fd:
            rd = csv.reader(fd, delimiter="\t", quotechar='"')
            for row in rd:
                if row[0] == 'Merged':
                    # Converting string to list 
                    res = row[1].strip('][').split(', ')
                    file.append(res)
        if len(file) < 1:
            i.append(doc)
        else:
            len_file = len(file[0])

            count = file[0].count("'"+word+"'")
            tf = round(count/len_file, 4)
            idf = np.log(len_dict/len(val_word))
            if word in New_Dict.keys():
                New_Dict[word].append((doc, tf*idf))
            else:
                New_Dict[word] = [(doc, tf*idf)]

### Saving the inverted matrix

In [15]:
with open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/tfidf_DICT.json', 'w') as outfile:
    json.dump(New_Dict, outfile)

### Load the dictionary with the inverted matrix

In [90]:
f =  open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/tfidf_DICT.json', 'r')
New_Dict = json.load(f)

### Stemming the query

In [91]:
# function for stemming 
def stem_word(lst_words):
    clean_lst_words= []
    for word in lst_words:
        word = ps.stem(word)
        clean_lst_words.append(word)
    return(clean_lst_words)
query = ['Disney', 'movie', 'love']
query = stem_word(query)

### Query TF-IDF

In [95]:
def tf_idf_query(query):
    tf_idf = {}
    len_query = len(query)
    len_dict = len(os.listdir(path))
    DF = {}
    for word in query:# its a count funtion
        if word not in DF.keys():
            DF[word] = 1 
        else:
            DF[word]+=1
    
    for word in query:
        count = query.count(word)
        tf =round(count/len_query, 4)
        val_word = DF[word]
        idf = np.log(len(DF.keys())/val_word)
        if word in tf_idf.keys():
            tf_idf[word]+= tf*idf
        else:
            tf_idf[word] = tf*idf
    return tf_idf

In [111]:
tf_idf_query = tf_idf_query(query)  

TypeError: 'dict' object is not callable

## 2.2.2) Execute the query

### Ranking - Cosine Similarity

In [97]:
import math

def numerator(query, doc):
    sum_ = []
    for word in query:
        for docs in New_Dict[word]:
            if doc == docs[0]:
                val_query = tf_idf_query[word]
                val_word = docs[1]
                molt_word = val_query * val_word
                sum_.append(molt_word)
    return(sum(sum_))
                
def norm_doc(query, doc):
    tot_sum = 0
    for i in range(len(query)):
        sum_ = 0
        for docs in New_Dict[query[i]]:
            if doc == docs[0]:
                val = (docs[1])**2
                sum_ = val
        tot_sum += sum_
    return(math.sqrt(tot_sum))

def norm_query(query):
    tot_sum = 0
    for i in range(len(query)):
        val = tf_idf_query[query[i]]
        val = val**2
        tot_sum += val
    return(math.sqrt(tot_sum))


In [108]:
L = []
for i in result:
    num = numerator(query, i)
    denom = norm_doc(query, i) * norm_query(query)
    L.append([i, round((num / denom)*100, 3)])

In [109]:
cos_similarity = sorted(L,key = lambda element: element[1],reverse = True)

In [110]:
to_df = []
for movie in cos_similarity:
    element_in_movie = {}
    with open(path+movie[0]) as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"')
        for row in rd:
            if row[0] == 'Name':
                element_in_movie['Name'] = row[1]
            elif row[0] == 'Intro':
                element_in_movie['Intro'] = row[1]
            elif row[0] == 'Url':
                element_in_movie['Url'] = row[1]
        element_in_movie['Similarity'] = str(movie[1])+' %'
    to_df.append(element_in_movie)
df = pd.DataFrame(to_df)
df = df.reindex(('Name', 'Intro', 'Url', 'Similarity'), axis=1)
df.iloc[:10]

,Name,Intro,Url,Similarity
0,The Love Bug,The Love Bug (sometimes referred to as Herbie ...,https://en.wikipedia.org/wiki/The_Love_Bug,95.186 %
1,Herbie Rides Again,Herbie Rides Again is a 1974 American comedy f...,https://en.wikipedia.org/wiki/Herbie_Rides_Again,93.237 %
2,Invitation to the Dance,Invitation to the Dance is a 1956 anthology fi...,https://en.wikipedia.org/wiki/Invitation_to_th...,90.74 %
3,Darby O'Gill and the Little People,Darby O'Gill and the Little People is a 1959 W...,https://en.wikipedia.org/wiki/Darby_O%27Gill_a...,87.316 %
4,Herbie Goes to Monte Carlo,Herbie Goes to Monte Carlo is a 1977 American ...,https://en.wikipedia.org/wiki/Herbie_Goes_to_M...,87.316 %
5,Bambi,Bambi is a 1942 American animated film directe...,https://en.wikipedia.org/wiki/Bambi,74.218 %
6,The Three Caballeros,The Three Caballeros is a 1944 American live-a...,https://en.wikipedia.org/wiki/The_Three_Caball...,70.21 %
7,Summer Magic,Summer Magic is a 1963 Walt Disney Productions...,https://en.wikipedia.org/wiki/Summer_Magic,70.21 %
